In [1]:
import csv
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from utils import *
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import tensorflow as tf
import tensorflowjs as tfjs
from preprocess import Preprocessor
from train import *

_model_dir = 'model'

2024-09-17 00:43:03.460080: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 00:43:03.472462: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 00:43:04.109783: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 00:43:07.984903: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 00:43:10.521402: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

0
0
0
0


In [2]:
train= make_df("dataset", "train")
train.head

  0%|          | 0/816 [00:00<?, ?it/s]

100%|██████████| 324/324 [00:04<00:00, 79.25it/s]

['Skipped 0.jpg:Zone.Identifier - Invalid file type', 'Skipped 1.jpg:Zone.Identifier - Invalid file type', 'Skipped 10.jpg:Zone.Identifier - Invalid file type', 'Skipped 11.jpg:Zone.Identifier - Invalid file type', 'Skipped 12.jpg:Zone.Identifier - Invalid file type', 'Skipped 13.jpg:Zone.Identifier - Invalid file type', 'Skipped 14.jpg:Zone.Identifier - Invalid file type', 'Skipped 15.jpg:Zone.Identifier - Invalid file type', 'Skipped 16.jpg:Zone.Identifier - Invalid file type', 'Skipped 17.jpg:Zone.Identifier - Invalid file type', 'Skipped 18.jpg:Zone.Identifier - Invalid file type', 'Skipped 19.jpg:Zone.Identifier - Invalid file type', 'Skipped 2.jpg:Zone.Identifier - Invalid file type', 'Skipped 20.jpg:Zone.Identifier - Invalid file type', 'Skipped 21.jpg:Zone.Identifier - Invalid file type', 'Skipped 22.jpg:Zone.Identifier - Invalid file type', 'Skipped 23.jpg:Zone.Identifier - Invalid file type', 'Skipped 24.jpg:Zone.Identifier - Invalid file type', 'Skipped 25.jpg:Zone.Identifie

<bound method NDFrame.head of     filename  NOSE_x  NOSE_y  NOSE_score  LEFT_EYE_x  LEFT_EYE_y  \
0      0.jpg    61.0    58.0    0.536201        56.0        54.0   
1      1.jpg    60.0    54.0    0.583177        54.0        51.0   
2     10.jpg    98.0    55.0    0.588507        93.0        50.0   
3     11.jpg   114.0    67.0    0.540989       111.0        62.0   
4     12.jpg   118.0    80.0    0.504765       114.0        75.0   
..       ...     ...     ...         ...         ...         ...   
666   95.jpg    63.0    27.0    0.696270        68.0        23.0   
667   96.jpg    68.0    26.0    0.716201        72.0        23.0   
668   97.jpg    74.0    29.0    0.526421        77.0        24.0   
669   98.jpg    81.0    30.0    0.711263        84.0        26.0   
670   99.jpg    86.0    30.0    0.605774        89.0        26.0   

     LEFT_EYE_score  RIGHT_EYE_x  RIGHT_EYE_y  RIGHT_EYE_score  ...  \
0          0.577895         61.0         54.0         0.583147  ...   
1          

In [3]:
X, y, class_names = load_data(train)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# Second split: validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [4]:
processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)
# print("process x train", processed_X_train)
# print("process x test", processed_X_test)

inputs = tf.keras.Input(shape=(34,))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)


In [5]:
model = keras.Model(inputs, outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
# Update checkpoint path to use the .keras extension
checkpoint_path = "weights.best.keras"

checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                             monitor='val_accuracy',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='max')

earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)


In [6]:
# Start training
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=200,
                    batch_size=16,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])


print('-----------------EVALUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)


tfjs.converters.save_keras_model(model, tfjs_model_dir)
print('tfjs model saved at ',tfjs_model_dir)


--------------TRAINING----------------
Epoch 1/200
17/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4123 - loss: 1.0653
Epoch 1: val_accuracy improved from -inf to 0.57426, saving model to weights.best.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.4515 - loss: 1.0380 - val_accuracy: 0.5743 - val_loss: 0.8595
Epoch 2/200
23/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6040 - loss: 0.8483 
Epoch 2: val_accuracy improved from 0.57426 to 0.80198, saving model to weights.best.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6040 - loss: 0.8407 - val_accuracy: 0.8020 - val_loss: 0.6800
Epoch 3/200
24/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7504 - loss: 0.7065 
Epoch 3: val_accuracy improved from 0.80198 to 0.93069, saving model to weights.best.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7551 - loss: 0.6966 - val_accuracy: 0.9307 - val_loss: 0.4993
Epoch 4/200
27/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8596 - loss: 0.5409 
Ep

LOSS:  0.005881540477275848
ACCURACY:  1.0
failed to lookup keras version from the file,
    this is likely a weight only file
tfjs model saved at  model
